## 1st Iteration
- Run on Sample(1000) to understand key themes

### Data & Imports

In [110]:
import pandas as pd
import re
import json
import os
import math
from openai import OpenAI
from dotenv import load_dotenv
import time

In [111]:
df = pd.read_csv("Project_Sandals_Google_Reviews_V2.csv")
df['reviews_id'] = df.index + 1
df.dropna(subset='review_text', inplace=True)
# df = df.sample(10)
# reviews_df = df[['review_text','reviews_id']]

In [112]:
df = df[df['reviews_id'].isin([10713, 1013, 14431, 2545, 10050])]

In [113]:
reviews_df = df[['review_text','reviews_id']]

In [114]:
def clean_review(text):
    text = re.sub(r"<.*?>", "", text)
    text = text.replace("\n", " ").strip()
    return text

reviews_df['review_text'] = reviews_df['review_text'].apply(clean_review)

C:\Users\prjain\AppData\Local\Temp\ipykernel_20608\2039213048.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['review_text'] = reviews_df['review_text'].apply(clean_review)


In [115]:
reviews_df['sentiment'] = None
reviews_df['topic'] = None
reviews_df['support_text'] = None
reviews_df['translated_review_text'] = None

C:\Users\prjain\AppData\Local\Temp\ipykernel_20608\2612990079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['sentiment'] = None
C:\Users\prjain\AppData\Local\Temp\ipykernel_20608\2612990079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['topic'] = None
C:\Users\prjain\AppData\Local\Temp\ipykernel_20608\2612990079.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [116]:
reviews_df

,review_text,reviews_id,sentiment,topic,support_text,translated_review_text
1012,"Wow, what a great location. Had butler servic...",1013,None,None,None,None
2544,Our 2nd Sandals Trip (1st was Exuma last year)...,2545,None,None,None,None
10049,"As a Marriott Loyalist, this property should n...",10050,None,None,None,None
10712,The place was not a good representation of Ant...,10713,None,None,None,None
14430,"Stunning hotel and grounds, what's not to like...",14431,None,None,None,None


### Batch Processing

#### create input

In [119]:
MAX_REQUESTS_PER_BATCH = 50000
MAX_BATCH_SIZE_BYTES = 190 * 1024 * 1024  #190mb

def build_request_line(review_id, review_text):
    system_prompt = (
    "You are a helpful assistant that analyzes customer reviews.\n"
    "For each review:\n"
    "- If the review is not in English, translate it to English first and include the translated text.\n"
    "  If it's already in English, keep it as is and set translated_text to the original.\n"
    "- Use the English version of the review to:\n"
    "  - Determine the sentiment (Positive, Neutral, or Negative).\n"
    "  - Identify a short topic label from the following list:\n"
    "  - Extract a short **exact phrase** (verbatim) from the English text that supports the topic.\n\n"
    "Return ONLY a strict JSON object with the following format (no explanations, no extra text):\n"
    '{"translated_text": "...", "sentiment": "...", "topic": "...", "support_text": "..."}'
    )

    
    request_body = {
        "custom_id": str(review_id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Review: {review_text}"}
            ],
            "max_tokens": 1024,         #increased token length as getting None in sentiment, topic & support text
            "temperature": 0.0
        }
    }
    return json.dumps(request_body)


batch_num = 1
batch_lines = []
batch_size_bytes = 0

for idx, row in reviews_df.iterrows():
    line = build_request_line(row['reviews_id'], row['review_text']) + "\n"
    line_size = len(line.encode('utf-8'))

    # Check if adding this line exceeds limits
    if (len(batch_lines) >= MAX_REQUESTS_PER_BATCH) or (batch_size_bytes + line_size > MAX_BATCH_SIZE_BYTES):
        # Write current batch to file
        batch_file = f"batchinput_part{batch_num}.jsonl"
        with open(batch_file, "w", encoding="utf-8") as f:
            f.writelines(batch_lines)
        print(f"Saved batch {batch_num} with {len(batch_lines)} lines ({batch_size_bytes / (1024*1024):.2f} MB) to '{batch_file}'")

        # Reset for next batch
        batch_num += 1
        batch_lines = []
        batch_size_bytes = 0

    # Add current line to batch
    batch_lines.append(line)
    batch_size_bytes += line_size

# Save the last batch if any lines left
if batch_lines:
    batch_file = f"batchinput_part{batch_num}.jsonl"
    with open(batch_file, "w", encoding="utf-8") as f:
        f.writelines(batch_lines)
    print(f"Saved batch {batch_num} with {len(batch_lines)} lines ({batch_size_bytes / (1024*1024):.2f} MB) to '{batch_file}'")


Saved batch 1 with 5 lines (0.01 MB) to 'batchinput_part1.jsonl'


#### upload & create batch

In [120]:
[f"batchinput_part{i}.jsonl" for i in range(1, batch_num + 1)]

['batchinput_part1.jsonl']

In [121]:
load_dotenv()
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

batch_job_ids = []

# Assume batch_num is the number of batches you created before
batch_files = [f"batchinput_part{i}.jsonl" for i in range(1, batch_num + 1)]

for bf in batch_files:
    # Upload the batch file
    with open(bf, "rb") as f:
        upload_response = openai.files.create(
            file=f,
            purpose="batch",
        )
    input_file_id = upload_response.id
    print(f"Uploaded {bf} with file id: {input_file_id}")

    # Create a batch job for this uploaded file
    batch_response = openai.batches.create(
    input_file_id=input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    )

    print(f"Created batch job for {bf} with job id: {batch_response.id}")
    batch_job_ids.append(batch_response.id)

Uploaded batchinput_part1.jsonl with file id: file-HcmnB7wd7o6qLXv1UAgSVg
Created batch job for batchinput_part1.jsonl with job id: batch_687895fc0fd881908c797ce9544f4337


#### poll, download & merge output

In [122]:
def poll_and_download(batch_id):
    print(f"Polling batch job {batch_id}...")
    while True:
        batch_info = openai.batches.retrieve(batch_id)
        status = batch_info.status
        print(f"Status of batch {batch_id}: {status}")
        if status in ["completed", "failed", "cancelled", "expired"]:
            break
        time.sleep(10)

    if status != "completed":
        print(f"Batch {batch_id} ended with status {status}. Skipping download.")
        return {}

    output_file_id = batch_info.output_file_id
    print(f"Downloading results for batch {batch_id} with file id {output_file_id}...")

    output_file_response = openai.files.content(output_file_id)
    output_lines = output_file_response.text.splitlines()

    results = {}
    for line in output_lines:
        try:
            result_json = json.loads(line)
            custom_id = result_json.get("custom_id")
            response_body = result_json.get("response", {}).get("body", {})
            choices = response_body.get("choices", [])
            if choices:
                content = choices[0]["message"]["content"]
                try:
                    parsed_content = json.loads(content)
                except json.JSONDecodeError:
                    parsed_content = {
                        "error": "Invalid JSON in GPT response",
                        "raw_content": content
                    }
                results[custom_id] = parsed_content
            else:
                results[custom_id] = {"error": "No choices in response"}
        except Exception as e:
            print(f"Error parsing line in batch {batch_id}: {e}")
    return results


In [123]:
all_results = {}
for job_id in batch_job_ids:
    batch_results = poll_and_download(job_id)
    all_results.update(batch_results)

print(f"Total results fetched: {len(all_results)}")

for col in ['sentiment', 'topic', 'support_text']:
    if col not in reviews_df.columns:
        reviews_df[col] = None

for cid, res in all_results.items():
    if "error" in res:
        continue  # skip errored entries
    
    sentiment = res.get("sentiment")
    topic = res.get("topic")
    support_text = res.get("support_text")
    translated_text = res.get("translated_text")

    mask = reviews_df['reviews_id'].astype(str) == cid
    
    # Update sentiment, topic, support_text
    reviews_df.loc[mask, 'sentiment'] = sentiment
    reviews_df.loc[mask, 'topic'] = topic
    reviews_df.loc[mask, 'support_text'] = support_text

    # Update review_text if translated_text is present and different
    if translated_text and translated_text.strip() and translated_text.strip() != reviews_df.loc[mask, 'review_text'].values[0].strip():
        reviews_df.loc[mask, 'review_text'] = translated_text.strip()

Polling batch job batch_687895fc0fd881908c797ce9544f4337...
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: in_progress
Status of batch batch_687895fc0fd881908c797ce9544f4337: completed
Total results fetched: 5


#### post processing

In [124]:
reviews_df

,review_text,reviews_id,sentiment,topic,support_text,translated_review_text
1012,"Wow, what a great location. Had butler servic...",1013,Positive,Service,"Nick, Navarro, and Delano were fantastic.",None
2544,Our 2nd Sandals Trip (1st was Exuma last year)...,2545,Positive,Service,Dillon was OUTSTANDING!,None
10049,"As a Marriott Loyalist, this property should n...",10050,Neutral,Property Maintenance,"the rooms, pool areas, cabanas, need routine m...",None
10712,The place was not a good representation of Ant...,10713,Negative,Service Quality,"Terrible service, unclean facilities",None
14430,"Stunning hotel and grounds, what's not to like...",14431,Positive,Hotel Experience,Stunning hotel and grounds,None


In [109]:
reviews_df['review_text'].loc[10049]

'As a Marriott Loyalist, this property should not be considered a 5-star resorts per Marriott standards.  It is a property that has not yet found its identity.  I would rate it 3.5 stars with a 5-star designation for the bungalows and the availability of the Diamond club services. It is a small property, not as large as what one would think of when using the word resort.  Although the landscaping, is well maintained, the rooms, pool areas, cabanas, need routine maintenance, if addressed in a timely manner, these would not be major issues.   Rooms do have a slight musty smell, and one guest on our floor did complain about a sewage smell, but I believed they were able to clean out their shower drain and it took care of the problem.   Perhaps having repair cards available in rooms where guest can alert the property to minor repairs before they become major, could assist them.  The Food in the specialty restaurants was ok and although I had no problem eating in the buffet, it was basically

In [76]:
reviews_df.sample(5)

,review_text,reviews_id,sentiment,topic,support_text,translated_review_text
10712,The place was not a good representation of Ant...,10713,Negative,Service and cleanliness,"Terrible service, unclean facilities",None
1012,"Wow, what a great location. Had butler servic...",1013,Positive,Great service and food,Food is the best at all the Sandals resorts.,None
14430,"Stunning hotel and grounds, what's not to like...",14431,Positive,Hotel experience,Stunning hotel and grounds,None
2544,Our 2nd Sandals Trip (1st was Exuma last year)...,2545,None,None,None,None
10049,"As a Marriott Loyalist, this property should n...",10050,None,None,None,None


In [85]:
# check above : why getting None in sentiment, topic, support_text & translated_review_text
# bcoz review_text is very very long hitting token limit -- we have to truncate long texts for gpt-4o-mini (8k) or use gpt-4o (32k)
# check above rows with gpt-4o if working

In [ ]:
# temp_df = reviews_df[reviews_df['reviews_id'].isin([10713, 1013, 14431, 2545, 10050])]
# temp_df.to_csv("error_rows.csv", index=False, encoding='utf-8-sig')

In [80]:
reviews_df['review_text'].loc[10049]

'As a Marriott Loyalist, this property should not be considered a 5-star resorts per Marriott standards.  It is a property that has not yet found its identity.  I would rate it 3.5 stars with a 5-star designation for the bungalows and the availability of the Diamond club services. It is a small property, not as large as what one would think of when using the word resort.  Although the landscaping, is well maintained, the rooms, pool areas, cabanas, need routine maintenance, if addressed in a timely manner, these would not be major issues.   Rooms do have a slight musty smell, and one guest on our floor did complain about a sewage smell, but I believed they were able to clean out their shower drain and it took care of the problem.   Perhaps having repair cards available in rooms where guest can alert the property to minor repairs before they become major, could assist them.  The Food in the specialty restaurants was ok and although I had no problem eating in the buffet, it was basically

In [67]:
reviews_df['topic'].value_counts()

topic
Great service and entertainment    1
Hotel experience                   1
Service and cleanliness            1
Overall Satisfaction               1
Anniversary trip experience        1
Great service and food             1
Relaxation                         1
Resort experience                  1
Name: count, dtype: int64

In [68]:
# extract key themes from this sample
# run that list of themes on entire dataset

In [69]:
reviews_df['topic'].dropna().unique().tolist()

['Great service and entertainment',
 'Hotel experience',
 'Service and cleanliness',
 'Overall Satisfaction',
 'Anniversary trip experience',
 'Great service and food',
 'Relaxation',
 'Resort experience']

In [70]:
###chatgpt prompt to create a concise list using above list from : reviews_df['topic'].dropna().unique().tolist()

# You are a data analyst assisting with text clustering.

# Given a list of specific review topic phrases, your task is to:
# 1. Identify common semantic themes shared across the list.
# 2. Merge overlapping topics and group them under broader, high-level categories.
# 3. Return a concise list of unique, generalized topic labels (each 1–3 words long). 
# 4. Avoid repeating similar concepts with different wording — generalize wherever possible.
# 5. Output only a clean Python list of the merged topics.

# Input:
# <input list>

# Return:
# A Python list named `broader_topics` containing the merged, high-level categories.

## Iteration 2: Run full dataset using predefined topic list

In [ ]:
import pandas as pd
import re
import json
import os
import math
from openai import OpenAI
from dotenv import load_dotenv
import time


# broader_topics = []


df = pd.read_csv("Project_Sandals_Google_Reviews_V2.csv")
df['reviews_id'] = df.index + 1
df.dropna(subset='review_text', inplace=True)
df = df.sample(5000)
reviews_df = df[['review_text','reviews_id']]


def clean_review(text):
    text = re.sub(r"<.*?>", "", text)
    text = text.replace("\n", " ").strip()
    return text


reviews_df['review_text'] = reviews_df['review_text'].apply(clean_review)


reviews_df['sentiment'] = None
reviews_df['topic'] = None
reviews_df['support_text'] = None
reviews_df['translated_review_text'] = None


MAX_REQUESTS_PER_BATCH = 50000
MAX_BATCH_SIZE_BYTES = 190 * 1024 * 1024  #190mb

def build_request_line(review_id, review_text):
    system_prompt = (
        "You are a helpful assistant that analyzes customer reviews.\n"
        "For each review:\n"
        "- If the review is not in English, translate it to English first and include the translated text.\n"
        "  If it's already in English, keep it as is and set translated_text to the original.\n"
        "- Use the English version of the review to:\n"
        "  - Determine the sentiment (Positive, Neutral, or Negative).\n"
        f"  - Identify a short topic label from the following list: {', '.join(broader_topics)}.\n"
        "  - Extract a short **exact phrase** (verbatim) from the English text that supports the topic.\n\n"
        "Return your answer in this JSON format:\n"
        '{"translated_text": "...", "sentiment": "...", "topic": "...", "support_text": "..."}'
    )
    
    request_body = {
        "custom_id": str(review_id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Review: {review_text}"}
            ],
            "max_tokens": 150,
            "temperature": 0.0
        }
    }
    return json.dumps(request_body)


batch_num = 1
batch_lines = []
batch_size_bytes = 0

for idx, row in reviews_df.iterrows():
    line = build_request_line(row['reviews_id'], row['review_text']) + "\n"
    line_size = len(line.encode('utf-8'))

    # Check if adding this line exceeds limits
    if (len(batch_lines) >= MAX_REQUESTS_PER_BATCH) or (batch_size_bytes + line_size > MAX_BATCH_SIZE_BYTES):
        # Write current batch to file
        batch_file = f"batchinput_part{batch_num}.jsonl"
        with open(batch_file, "w", encoding="utf-8") as f:
            f.writelines(batch_lines)
        print(f"Saved batch {batch_num} with {len(batch_lines)} lines ({batch_size_bytes / (1024*1024):.2f} MB) to '{batch_file}'")

        # Reset for next batch
        batch_num += 1
        batch_lines = []
        batch_size_bytes = 0

    # Add current line to batch
    batch_lines.append(line)
    batch_size_bytes += line_size

# Save the last batch if any lines left
if batch_lines:
    batch_file = f"batchinput_part{batch_num}.jsonl"
    with open(batch_file, "w", encoding="utf-8") as f:
        f.writelines(batch_lines)
    print(f"Saved batch {batch_num} with {len(batch_lines)} lines ({batch_size_bytes / (1024*1024):.2f} MB) to '{batch_file}'")



load_dotenv()
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

batch_job_ids = []

# Assume batch_num is the number of batches you created before
batch_files = [f"batchinput_part{i}.jsonl" for i in range(1, batch_num + 1)]

for bf in batch_files:
    # Upload the batch file
    with open(bf, "rb") as f:
        upload_response = openai.files.create(
            file=f,
            purpose="batch",
        )
    input_file_id = upload_response.id
    print(f"Uploaded {bf} with file id: {input_file_id}")

    # Create a batch job for this uploaded file
    batch_response = openai.batches.create(
    input_file_id=input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    )

    print(f"Created batch job for {bf} with job id: {batch_response.id}")
    batch_job_ids.append(batch_response.id)



def poll_and_download(batch_id):
    print(f"Polling batch job {batch_id}...")
    while True:
        batch_info = openai.batches.retrieve(batch_id)
        status = batch_info.status
        print(f"Status of batch {batch_id}: {status}")
        if status in ["completed", "failed", "cancelled", "expired"]:
            break
        time.sleep(10)

    if status != "completed":
        print(f"Batch {batch_id} ended with status {status}. Skipping download.")
        return {}

    output_file_id = batch_info.output_file_id
    print(f"Downloading results for batch {batch_id} with file id {output_file_id}...")

    output_file_response = openai.files.content(output_file_id)
    output_lines = output_file_response.text.splitlines()

    results = {}
    for line in output_lines:
        try:
            result_json = json.loads(line)
            custom_id = result_json.get("custom_id")
            response_body = result_json.get("response", {}).get("body", {})
            choices = response_body.get("choices", [])
            if choices:
                content = choices[0]["message"]["content"]
                try:
                    parsed_content = json.loads(content)
                except json.JSONDecodeError:
                    parsed_content = {
                        "error": "Invalid JSON in GPT response",
                        "raw_content": content
                    }
                results[custom_id] = parsed_content
            else:
                results[custom_id] = {"error": "No choices in response"}
        except Exception as e:
            print(f"Error parsing line in batch {batch_id}: {e}")
    return results



all_results = {}
for job_id in batch_job_ids:
    batch_results = poll_and_download(job_id)
    all_results.update(batch_results)

print(f"Total results fetched: {len(all_results)}")

for col in ['sentiment', 'topic', 'support_text']:
    if col not in reviews_df.columns:
        reviews_df[col] = None

for cid, res in all_results.items():
    if "error" in res:
        continue  # skip errored entries
    
    sentiment = res.get("sentiment")
    topic = res.get("topic")
    support_text = res.get("support_text")
    translated_text = res.get("translated_text")

    mask = reviews_df['reviews_id'].astype(str) == cid
    
    # Update sentiment, topic, support_text
    reviews_df.loc[mask, 'sentiment'] = sentiment
    reviews_df.loc[mask, 'topic'] = topic
    reviews_df.loc[mask, 'support_text'] = support_text

    # Update review_text if translated_text is present and different
    if translated_text and translated_text.strip() and translated_text.strip() != reviews_df.loc[mask, 'review_text'].values[0].strip():
        reviews_df.loc[mask, 'review_text'] = translated_text.strip()

## Extract broader topic list from openAI also & then pass it so everything will be automated